In [279]:
from sympy import *
from sympy.matrices import BlockMatrix, Matrix, zeros
import sympy as sp
import numpy as np

In [280]:
Ts = 0.09
dt = 0.01
foot_height_keep_STF = 0.2
N = int(Ts/dt)
print('Nb steps = ', N)
nb_total_variables_per_coordinate = int(3 * N) # px, vx, zx
print('nb_total_variables_per_coordinate', nb_total_variables_per_coordinate)
nb_total_variables = 3 * nb_total_variables_per_coordinate # x, y, z
print(nb_total_variables, 'variables')

Nb steps =  9
nb_total_variables_per_coordinate 27
81 variables


In [281]:
variables = []
coord = ['x', 'y', 'z']
for c in coord:
    for i in range(N):
        p = symbols(f'p{c}{i}')
        v = symbols(f'v{c}{i}')
        a = symbols(f'a{c}{i}')
        variables.extend([p, v, a])
pva_vector = Matrix(variables)
print('------------------- Optimization Variable -------------------')
print('pva_vector = ', len(pva_vector), ' variables')
for i in range(3):
    pprint(pva_vector.T[i*nb_total_variables_per_coordinate:(i+1)*nb_total_variables_per_coordinate])


------------------- Optimization Variable -------------------
pva_vector =  81  variables
[px₀, vx₀, ax₀, px₁, vx₁, ax₁, px₂, vx₂, ax₂, px₃, vx₃, ax₃, px₄, vx₄, ax₄, px
₅, vx₅, ax₅, px₆, vx₆, ax₆, px₇, vx₇, ax₇, px₈, vx₈, ax₈]
[py₀, vy₀, ay₀, py₁, vy₁, ay₁, py₂, vy₂, ay₂, py₃, vy₃, ay₃, py₄, vy₄, ay₄, py
₅, vy₅, ay₅, py₆, vy₆, ay₆, py₇, vy₇, ay₇, py₈, vy₈, ay₈]
[pz₀, vz₀, az₀, pz₁, vz₁, az₁, pz₂, vz₂, az₂, pz₃, vz₃, az₃, pz₄, vz₄, az₄, pz
₅, vz₅, az₅, pz₆, vz₆, az₆, pz₇, vz₇, az₇, pz₈, vz₈, az₈]


In [282]:
print('------------------- P Mat -------------------')
opt_weight_desired_pos = 10
opt_weight_desired_vel = 1
zero_matrix_3_by_3 = Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 0]])

p_N_des = sp.Matrix([0.2, 0.3, 0.1])
v_N_des = sp.zeros(3)

q = zeros(nb_total_variables, 1)
P_matrix = zeros(nb_total_variables, nb_total_variables)

for i in range(nb_total_variables_per_coordinate):
    P_matrix[i * 3 + 2, i * 3 + 2] = 1.0

for i in range(3):
    P_matrix[(i + 1) * nb_total_variables_per_coordinate - 3, (i + 1) * nb_total_variables_per_coordinate - 3] = opt_weight_desired_pos
    P_matrix[(i + 1) * nb_total_variables_per_coordinate - 2, (i + 1) * nb_total_variables_per_coordinate - 2] = opt_weight_desired_vel

    q[(i+1) * nb_total_variables_per_coordinate - 3] = -2 * opt_weight_desired_pos * p_N_des[i]
    q[(i+1) * nb_total_variables_per_coordinate - 2] = -2 * opt_weight_desired_vel * v_N_des[i]

vPv = pva_vector.T * P_matrix * pva_vector

objective_function = vPv + q.T * pva_vector
print('Objective function')
pprint(objective_function)


------------------- P Mat -------------------
Objective function
⎡       2          2          2          2          2          2          2   
⎣1.0⋅ax₀  + 1.0⋅ax₁  + 1.0⋅ax₂  + 1.0⋅ax₃  + 1.0⋅ax₄  + 1.0⋅ax₅  + 1.0⋅ax₆  + 

       2          2          2          2          2          2          2    
1.0⋅ax₇  + 1.0⋅ax₈  + 1.0⋅ay₀  + 1.0⋅ay₁  + 1.0⋅ay₂  + 1.0⋅ay₃  + 1.0⋅ay₄  + 1

      2          2          2          2          2          2          2     
.0⋅ay₅  + 1.0⋅ay₆  + 1.0⋅ay₇  + 1.0⋅ay₈  + 1.0⋅az₀  + 1.0⋅az₁  + 1.0⋅az₂  + 1.

     2          2          2          2          2          2         2       
0⋅az₃  + 1.0⋅az₄  + 1.0⋅az₅  + 1.0⋅az₆  + 1.0⋅az₇  + 1.0⋅az₈  + 10⋅px₈  - 4.0⋅

            2                   2                2      2      2⎤
px₈ + 10⋅py₈  - 6.0⋅py₈ + 10⋅pz₈  - 2.0⋅pz₈ + vx₈  + vy₈  + vz₈ ⎦


In [283]:

A_eq_pos_vel_des = sp.zeros(6, nb_total_variables)

A_eq_pos_vel_des[0, 0] = 1
A_eq_pos_vel_des[1, 1] = 1

A_eq_pos_vel_des[2, nb_total_variables_per_coordinate] = 1
A_eq_pos_vel_des[3, nb_total_variables_per_coordinate + 1] = 1

A_eq_pos_vel_des[4, 2*nb_total_variables_per_coordinate] = 1
A_eq_pos_vel_des[5, 2*nb_total_variables_per_coordinate + 1] = 1

result_vector = A_eq_pos_vel_des*pva_vector
print('------------------- Boundary Points Equality Contraints -------------------')
print('A_eq_pos_vel_desired * v = ')
pprint(result_vector)

------------------- Boundary Points Equality Contraints -------------------
A_eq_pos_vel_desired * v = 
⎡px₀⎤
⎢   ⎥
⎢vx₀⎥
⎢   ⎥
⎢py₀⎥
⎢   ⎥
⎢vy₀⎥
⎢   ⎥
⎢pz₀⎥
⎢   ⎥
⎣vz₀⎦


In [284]:
dt_symbol = symbols('dt')
block_dynamics = zeros(2, 6)
block_dynamics[0, 0] = 1
block_dynamics[0, 1] = dt_symbol
block_dynamics[0, 3] = -1
block_dynamics[1, 1] = 1
block_dynamics[1, 2] = dt_symbol
block_dynamics[1, 4] = -1
A_dynamics_per_coordinate = zeros(2 * N - 2, 3 * N)

j = 0
for i in range(int((2 * N - 2)/2)):
    A_dynamics_per_coordinate[j : j + 2, i * 3: i * 3 + 6] = block_dynamics
    j = j + 2

pva_vector_x_comp = pva_vector[0:nb_total_variables_per_coordinate, 0]

size_rows_A = A_dynamics_per_coordinate.shape[0]
size_cols_A = A_dynamics_per_coordinate.shape[1]
A_dynamics = zeros(3 * size_rows_A, nb_total_variables)

for i in range(3):
    A_dynamics[i * size_rows_A : i * size_rows_A + size_rows_A, i * size_cols_A : i * size_cols_A + size_cols_A] = A_dynamics_per_coordinate

mult = A_dynamics * pva_vector
print('------------------- Dynamic Contraints -------------------')
pprint(mult)

------------------- Dynamic Contraints -------------------
⎡dt⋅vx₀ + px₀ - px₁⎤
⎢                  ⎥
⎢ax₀⋅dt + vx₀ - vx₁⎥
⎢                  ⎥
⎢dt⋅vx₁ + px₁ - px₂⎥
⎢                  ⎥
⎢ax₁⋅dt + vx₁ - vx₂⎥
⎢                  ⎥
⎢dt⋅vx₂ + px₂ - px₃⎥
⎢                  ⎥
⎢ax₂⋅dt + vx₂ - vx₃⎥
⎢                  ⎥
⎢dt⋅vx₃ + px₃ - px₄⎥
⎢                  ⎥
⎢ax₃⋅dt + vx₃ - vx₄⎥
⎢                  ⎥
⎢dt⋅vx₄ + px₄ - px₅⎥
⎢                  ⎥
⎢ax₄⋅dt + vx₄ - vx₅⎥
⎢                  ⎥
⎢dt⋅vx₅ + px₅ - px₆⎥
⎢                  ⎥
⎢ax₅⋅dt + vx₅ - vx₆⎥
⎢                  ⎥
⎢dt⋅vx₆ + px₆ - px₇⎥
⎢                  ⎥
⎢ax₆⋅dt + vx₆ - vx₇⎥
⎢                  ⎥
⎢dt⋅vx₇ + px₇ - px₈⎥
⎢                  ⎥
⎢ax₇⋅dt + vx₇ - vx₈⎥
⎢                  ⎥
⎢dt⋅vy₀ + py₀ - py₁⎥
⎢                  ⎥
⎢ay₀⋅dt + vy₀ - vy₁⎥
⎢                  ⎥
⎢dt⋅vy₁ + py₁ - py₂⎥
⎢                  ⎥
⎢ay₁⋅dt + vy₁ - vy₂⎥
⎢                  ⎥
⎢dt⋅vy₂ + py₂ - py₃⎥
⎢                  ⎥
⎢ay₂⋅dt + vy₂ - vy₃⎥
⎢                  ⎥
⎢dt⋅vy₃ + py₃ - p

In [285]:
block = zeros(2, 3)
block[0, 1] = 1
block[1, 2] = 1
A_limits = zeros(nb_total_variables_per_coordinate * 2, nb_total_variables)

for i in range(nb_total_variables_per_coordinate):
    A_limits[2*i:2*(i+1), 3*i:3*(i+1)] = block

pprint((A_limits*pva_vector).T)

[vx₀  ax₀  vx₁  ax₁  vx₂  ax₂  vx₃  ax₃  vx₄  ax₄  vx₅  ax₅  vx₆  ax₆  vx₇  ax
₇  vx₈  ax₈  vy₀  ay₀  vy₁  ay₁  vy₂  ay₂  vy₃  ay₃  vy₄  ay₄  vy₅  ay₅  vy₆  
ay₆  vy₇  ay₇  vy₈  ay₈  vz₀  az₀  vz₁  az₁  vz₂  az₂  vz₃  az₃  vz₄  az₄  vz₅
  az₅  vz₆  az₆  vz₇  az₇  vz₈  az₈]


In [286]:
A_total = Matrix.vstack(A_eq_pos_vel_des,
                    A_dynamics,
                    A_limits)

print('A * PVA_VECTOR')
pprint(A_total*pva_vector)

A * PVA_VECTOR
⎡       px₀        ⎤
⎢                  ⎥
⎢       vx₀        ⎥
⎢                  ⎥
⎢       py₀        ⎥
⎢                  ⎥
⎢       vy₀        ⎥
⎢                  ⎥
⎢       pz₀        ⎥
⎢                  ⎥
⎢       vz₀        ⎥
⎢                  ⎥
⎢dt⋅vx₀ + px₀ - px₁⎥
⎢                  ⎥
⎢ax₀⋅dt + vx₀ - vx₁⎥
⎢                  ⎥
⎢dt⋅vx₁ + px₁ - px₂⎥
⎢                  ⎥
⎢ax₁⋅dt + vx₁ - vx₂⎥
⎢                  ⎥
⎢dt⋅vx₂ + px₂ - px₃⎥
⎢                  ⎥
⎢ax₂⋅dt + vx₂ - vx₃⎥
⎢                  ⎥
⎢dt⋅vx₃ + px₃ - px₄⎥
⎢                  ⎥
⎢ax₃⋅dt + vx₃ - vx₄⎥
⎢                  ⎥
⎢dt⋅vx₄ + px₄ - px₅⎥
⎢                  ⎥
⎢ax₄⋅dt + vx₄ - vx₅⎥
⎢                  ⎥
⎢dt⋅vx₅ + px₅ - px₆⎥
⎢                  ⎥
⎢ax₅⋅dt + vx₅ - vx₆⎥
⎢                  ⎥
⎢dt⋅vx₆ + px₆ - px₇⎥
⎢                  ⎥
⎢ax₆⋅dt + vx₆ - vx₇⎥
⎢                  ⎥
⎢dt⋅vx₇ + px₇ - px₈⎥
⎢                  ⎥
⎢ax₇⋅dt + vx₇ - vx₈⎥
⎢                  ⎥
⎢dt⋅vy₀ + py₀ - py₁⎥
⎢                  ⎥
⎢ay₀⋅dt + vy₀ - vy₁

In [287]:
# Limits
p0_desired = 0.0
v0_desired = 0.0

l_boundary_pts_per_coordinate = Matrix([p0_desired, v0_desired])
u_boundary_pts_per_coordinate = Matrix([p0_desired, v0_desired])


l_boundary_pts = Matrix.vstack(l_boundary_pts_per_coordinate, l_boundary_pts_per_coordinate, l_boundary_pts_per_coordinate)
u_boundary_pts = Matrix.vstack(u_boundary_pts_per_coordinate, u_boundary_pts_per_coordinate, u_boundary_pts_per_coordinate)

l_boundary_pts

Matrix([
[0],
[0],
[0],
[0],
[0],
[0]])

In [288]:
v_max = 20.0
a_max = 100.0
l_limits_per_coordinate = zeros(2 * N, 1)
u_limits_per_coordinate = zeros(2 * N, 1)

for i in range(0, 2 * N, 2):
    l_limits_per_coordinate[i, 0] = -v_max
    u_limits_per_coordinate[i, 0] = v_max

for i in range(1, 2 * N, 2):
    l_limits_per_coordinate[i, 0] = -a_max
    u_limits_per_coordinate[i, 0] = a_max


l_limits = Matrix.vstack(l_limits_per_coordinate, l_limits_per_coordinate, l_limits_per_coordinate)
u_limits = Matrix.vstack(u_limits_per_coordinate, u_limits_per_coordinate, u_limits_per_coordinate)
pprint(l_limits.T)
pprint(u_limits.T)


[-20.0  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100.0  -2
0.0  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0
  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -
100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100
.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100.0  -20.0  -100.0 
 -20.0  -100.0]
[20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0 
 20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0 
 20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0 
 20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0  20.0  100.0 
 20.0  100.0  20.0  100.0  20.0  100.0]


In [289]:

T_keep = 33/100*Ts
T_start_keep = 33/100*Ts
T_end_keep = T_keep + T_start_keep
n_keep = int(T_keep/dt)
n_start_keep = int(T_start_keep/dt)
n_end_keep = n_keep + n_start_keep

A_keep_foot = zeros(n_keep, nb_total_variables)

j = n_start_keep
for i in range(n_keep):
    A_keep_foot[i, 2*nb_total_variables_per_coordinate + 3*j] = 1
    j = j + 1

pprint(A_keep_foot @ pva_vector)

⎡pz₂⎤
⎢   ⎥
⎣pz₃⎦


In [290]:
pprint(pva_vector[0])
pprint(pva_vector[nb_total_variables_per_coordinate-3])

pprint(pva_vector[0:nb_total_variables_per_coordinate:3])

px₀
px₈
[px₀, px₁, px₂, px₃, px₄, px₅, px₆, px₇, px₈]
